<a href="https://colab.research.google.com/github/SZAftabi/UseRQE/blob/main/3_UserHistoryRetrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center> <font size='6'> 💟 <b> UseRQE </b> 💟 </font> <br> </center>
<center>Recognizing Question Entailment with User Background-knowledge Modeling <br> </center> <center> <font size='4' color='red'> <b> Step (3) </b> Users' history retrieval </font> </center>


# 😎 **Mount the drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
Drive_path = "/content/drive/MyDrive/"

# 😎 **1. Libraries**

In [ ]:
import os
import re
import torch
import warnings
import nltk
import json
import time
import requests
nltk.download('punkt')

import numpy as np
import pandas as pd
import bitsandbytes as bnb
import pytorch_lightning as pl
import matplotlib.pyplot as plt


# 😎 **2. Find first user's history**


In [ ]:
data_path_LLama = f"{Drive_path}RQE_Data.pkl"
MyData_LLama = pd.read_pickle(data_path_LLama)

In [ ]:
MyData_LLama.loc[(MyData_LLama['body_Q2']=='') & (MyData_LLama['userid_Q2']=='65001'), 'body_Q2']='So when I launch Minecraft, before it finishes loading, it crashes. I do not understand what is going on. Could someone help me? Here is my crash report:'
MyData_LLama.loc[(MyData_LLama['body_Q2']=='') & (MyData_LLama['userid_Q2']=='36896'), 'body_Q2']='How do I type the infinity symbol in MacTex'
MyData_LLama.loc[(MyData_LLama['body_Q2']=='') & (MyData_LLama['userid_Q2']=='3031'), 'body_Q2']='Run time error for GP objects'
MyData_LLama.loc[(MyData_LLama['body_Q1']=='') & (MyData_LLama['userid_Q2']=='65001'), 'body_Q1']='Misplaced allignment tab character line 53'
MyData_LLama.loc[(MyData_LLama['body_Q1']=='') & (MyData_LLama['userid_Q2']=='16188'), 'body_Q1']='How to Export this animation as a gif file for powerpoint presentation'
MyData_LLama.loc[(MyData_LLama['body_Q1']=='') & (MyData_LLama['userid_Q2']=='24829'), 'body_Q1']='why does rotation style work on actual coordinates and not variables in tikz 3d plot'

MyData_LLama.loc[(MyData_LLama['body_Q2']=='') & (MyData_LLama['userid_Q2']=='50615'), 'body_Q2']='How set a table in margin'
MyData_LLama.loc[(MyData_LLama['body_Q2']=='') & (MyData_LLama['userid_Q2']=='23835'), 'body_Q2']='Latex equation positioning problem'
MyData_LLama.loc[(MyData_LLama['body_Q2']=='') & (MyData_LLama['userid_Q2']=='14524'), 'body_Q2']='Chapter comment with regulation'
MyData_LLama.loc[(MyData_LLama['body_Q2']=='') & (MyData_LLama['userid_Q2']=='50823'), 'body_Q2']='minipage goes beyond right margin'

In [ ]:
user_history_df = pd.DataFrame(columns=['userid', 'historyCount', 'historyIDs', 'history'])
for index, row in MyData_LLama.iterrows():
    user_id = row['userid_Q1']
    forum_name = row['forum_x']

    AA = (MyData_LLama[
        (MyData_LLama['userid_Q1'] == user_id) & (MyData_LLama['forum_x'] == forum_name)
        ][['id_Q1', 'body_Q1']])
    AA.rename(
        columns={'id_Q1': 'QuestionID', 'body_Q1': 'QuestionBody'},
        inplace=True
        )
    BB = (MyData_LLama[
        (MyData_LLama['userid_Q2'] == user_id) & (MyData_LLama['forum_y'] == forum_name)
        ][['id_Q2', 'body_Q2']])
    BB.rename(
        columns={'id_Q2': 'QuestionID', 'body_Q2': 'QuestionBody'},
        inplace=True
        )
    user_history = pd.concat([AA, BB])
    user_history = user_history.drop_duplicates(subset=['QuestionID'])

    user_history_str = ', '.join(user_history['QuestionID'].astype(str))
    user_history_body_str = ', '.join(user_history['QuestionBody'].astype(str))

    user_history_df = pd.concat(
        [user_history_df, pd.DataFrame({'userid': [user_id],
                                        'forum': [forum_name],
                                        'historyCount': len(user_history),
                                        'historyIDs': [user_history_str],
                                        'history': [user_history_body_str]})
        ], ignore_index=True)

user_history_df_filepath = f"{Drive_path}UseRQE/TG/user_history_df.pkl"
user_history_df.to_pickle(user_history_df_filepath)
display(user_history_df)


# 😎 **3. Find second user's history**
appending them to user_history_df


In [ ]:
for index, row in MyData_LLama.iterrows():
    user_id = row['userid_Q2']
    forum_name = row['forum_y']

    AA = (MyData_LLama[
        (MyData_LLama['userid_Q1'] == user_id) & (MyData_LLama['forum_x'] == forum_name)
        ][['id_Q1', 'body_Q1']])
    AA.rename(
        columns={'id_Q1': 'QuestionID', 'body_Q1': 'QuestionBody'},
        inplace=True
        )
    BB = (MyData_LLama[
        (MyData_LLama['userid_Q2'] == user_id) & (MyData_LLama['forum_y'] == forum_name)
        ][['id_Q2', 'body_Q2']])
    BB.rename(
        columns={'id_Q2': 'QuestionID', 'body_Q2': 'QuestionBody'},
        inplace=True
        )
    user_history = pd.concat([AA, BB])
    user_history = user_history.drop_duplicates(subset=['QuestionID'])

    user_history_str = ', '.join(user_history['QuestionID'].astype(str))
    user_history_body_str = ', '.join(user_history['QuestionBody'].astype(str))

    user_history_df = pd.concat(
        [user_history_df, pd.DataFrame({'userid': [user_id],
                                        'forum': [forum_name],
                                        'historyCount': len(user_history),
                                        'historyIDs': [user_history_str],
                                        'history': [user_history_body_str]})
        ], ignore_index=True)

user_history_df_filepath = f"{Drive_path}ReQuEST/user_history_Both_df2.pkl"
user_history_df.to_pickle(user_history_df_filepath)
display(user_history_df)